In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None

Добрый день! В первой задаче возник вопрос о правильности применения groupby и mean. Конкретно,
в этом месте:
    
raitings_movies = df_ratings[['movieId', 'rating']]
raitings_movies2 = raitings_movies.groupby(['movieId']).mean()

Первый вариант отрабатывал некорректно, среднее считалось также, но после прогона через 
функцию по части значений класс определялся некорректно. Причем по непонятной логике. Часть фильмов с со средними значениями около 3.5 попадали в низкий рейтинг. На всякий случай в функции int-ы были 
заменены на float, хотя никакого влияния в рамках Питона это оказывать не должно. Конкрентно в данном случае.

Переделав логику работы с массивом следующим образом, проблему удалось устранить:
    
raitings_movies = raitings_movies.groupby('movieId')['rating'].mean()

Но вопрос о внутренней логике работы этих методов и причинах некорректного определения класса остался. Подскажите, пожалуйста, в чем может быть дело.

PS: отдельно отмечу, что inplace пока не использую сознательно) Чтобы лучше контролировать все этапы в "ручном" режиме.)


**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class


In [1]:
def class_of_ratings(ratings):

    if 0 <= ratings <= 2.0:
        return "низкий рейтинг"
    elif 2.0 <= ratings <= 4.0:
        return "средний рейтинг"
    elif 4.0 < ratings <= 5.0:
        return "высокий рейтинг"


In [4]:
df_movies = pd.read_csv('movies.csv')
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9120,162672,Mohenjo Daro (2016),Adventure|Drama|Romance
9121,163056,Shin Godzilla (2016),Action|Adventure|Fantasy|Sci-Fi
9122,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary
9123,164977,The Gay Desperado (1936),Comedy


In [6]:
df_ratings = pd.read_csv('ratings.csv')
df_ratings.sort_values('movieId')

,userId,movieId,rating,timestamp
9713,68,1,4.0,1194741818
35933,261,1,1.5,1101665532
52631,383,1,5.0,852806429
35983,262,1,2.5,1433898798
12038,77,1,4.0,1163005363
...,...,...,...,...
39546,287,161944,5.0,1470167824
11823,73,162376,4.5,1474255532
92339,611,162542,5.0,1471520667
92340,611,162672,3.0,1471523986


In [11]:
# Проблемные ячейки
# raitings_movies = df_ratings[['movieId', 'rating']]
# raitings_movies2 = raitings_movies.groupby(['movieId']).mean()

# raitings_movies2

raitings_movies = raitings_movies.groupby('movieId')['rating'].mean()
raitings_movies

movieId
1         3.872470
2         3.401869
3         3.161017
4         2.384615
5         3.267857
            ...   
161944    5.000000
162376    4.500000
162542    5.000000
162672    3.000000
163949    5.000000
Name: rating, Length: 9066, dtype: float64

In [12]:
# Проблемные ячейки
# raitings_movies2['class'] = raitings_movies.apply(class_of_ratings, axis=1)
# raitings_movies3 = raitings_movies2.reset_index()
# raitings_movies3.head()

raitings_movies2['class'] = raitings_movies.apply(class_of_ratings)
raitings_movies3 = raitings_movies2.reset_index()
raitings_movies3.head()

,movieId,rating,class
0,1,3.872470,средний рейтинг
1,2,3.401869,средний рейтинг
2,3,3.161017,средний рейтинг
3,4,2.384615,средний рейтинг
4,5,3.267857,средний рейтинг


In [13]:
summary_table = raitings_movies3.merge(df_movies, how='left').set_index('movieId')
summary_table.head()

,rating,class,title,genres
movieId,,,,
1,3.872470,средний рейтинг,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,3.401869,средний рейтинг,Jumanji (1995),Adventure|Children|Fantasy
3,3.161017,средний рейтинг,Grumpier Old Men (1995),Comedy|Romance
4,2.384615,средний рейтинг,Waiting to Exhale (1995),Comedy|Drama|Romance
5,3.267857,средний рейтинг,Father of the Bride Part II (1995),Comedy


In [16]:
columnsTitles = ['title', 'genres', 'class', 'rating']
summary_table = summary_table.reindex(columns = columnsTitles)
summary_table.head(10)

,title,genres,class,rating
movieId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг,3.872470
2,Jumanji (1995),Adventure|Children|Fantasy,средний рейтинг,3.401869
3,Grumpier Old Men (1995),Comedy|Romance,средний рейтинг,3.161017
4,Waiting to Exhale (1995),Comedy|Drama|Romance,средний рейтинг,2.384615
5,Father of the Bride Part II (1995),Comedy,средний рейтинг,3.267857
6,Heat (1995),Action|Crime|Thriller,средний рейтинг,3.884615
7,Sabrina (1995),Comedy|Romance,средний рейтинг,3.283019
8,Tom and Huck (1995),Adventure|Children,средний рейтинг,3.800000
9,Sudden Death (1995),Action,средний рейтинг,3.150000


**Задание 2**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

*Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:*
    
   geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}   


Результат классификации запишите в отдельный столбец region.


In [122]:
df_keywords = pd.read_csv('keywords.csv')
df_keywords

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195
...,...,...
99995,эльдорадо старый оскол,3705
99996,frigate для firefox,3630
99997,укрсиб,3630
99998,погода в ялте на неделю,3688


In [175]:
# функция проверяет наличие искомой подстроки в предложенной из датафрейма строке. 
# Изначально было желание поиграться с регулярками, но показалось, что здесь это будет избыточным 
# усложнением. Как и по объему предварительной работы, так и по ветвлению условий в самой функции.
# Ведь не получится написать одну на все варианты городов. С другой стороны, остались сомнения 
# относительно эффективности выбранного варианта поиска. И с точки зрения охвата всех 
# возможных вариантов, и с точки зрения времени работы таких циклов. Особенно при увеличении словаря.


def what_region_2(string):
    geo_data = { 'Центр': ['москва', 'тула', 'ярославль'],
                'Северо-Запад': ['петербург', 'псков', 'мурманск'],
                'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }
    
    for key, value in geo_data.items():
        for i in value:
            if i in string:
                return key
    return 'undefined'

In [176]:
df_keywords['region'] = df_keywords.keyword.apply(what_region_2)
df_keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [177]:
df_keywords_far_east = df_keywords[df_keywords['region'] == 'Дальний Восток']
df_keywords_far_east

,keyword,shows,region
1063,фарпост владивосток,176951,Дальний Восток
2175,дром хабаровск,98361,Дальний Восток
3206,дром владивосток,73084,Дальний Восток
3275,100 сп владивосток,76418,Дальний Восток
3687,авито хабаровск,63736,Дальний Восток
...,...,...,...
90402,аэропорт владивосток,4658,Дальний Восток
92442,монастырев владивосток официальный сайт,3870,Дальний Восток
92843,проститутки хабаровск,4120,Дальний Восток
93872,биглион хабаровск,3890,Дальний Восток


In [178]:
df_keywords_northwest = df_keywords[df_keywords['region'] == 'Северо-Запад']
df_keywords_northwest

,keyword,shows,region
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
1236,банк санкт петербург,174375,Северо-Запад
1346,санкт петербург,144232,Северо-Запад
2121,электронный дневник санкт петербургского образ...,94835,Северо-Запад
...,...,...,...
99084,122 медсанчасть санкт петербург официальный сайт,3679,Северо-Запад
99134,росреестр по санкт петербургу официальный сайт,3700,Северо-Запад
99204,bileter ru санкт петербург,3661,Северо-Запад
99634,северпост новости мурманской области,3606,Северо-Запад


In [180]:
df_keywords_center = df_keywords[df_keywords['region'] == 'Центр']
df_keywords_center

,keyword,shows,region
127,авито москва,979292,Центр
849,авито ярославль,209581,Центр
1259,москва,153050,Центр
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр
...,...,...,...
98943,вакансии на авито москва от прямых работодателей,3738,Центр
99292,афиша кино москва на сегодня,3704,Центр
99590,авито ярославль автомобили с пробегом,3673,Центр
99890,яндекс метро москва,4548,Центр
